In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os


In [ ]:
path = r"C:\Users\bashi\Documents\Sadia\US documents\SUNY albany\Charalampos Chelmis\CARES homeless project\Results\Wayfinding_firstApproach\20210527_Final Analysis for ASONAM\Final Results\Github"

In [ ]:
# Dataframe preparation
df = pd.read_csv(path+"\\ToyDataset.csv").drop(['Unnamed: 0'],axis=1)

clientID = df["ClientID"].unique()

In [ ]:
# separating the living situation and the exit destination into categories
stable = [2,3,10,11,12,13,14,15,19,20,21,22,23,25,26,27,28,31]
unstable = [1,4,5,6,7,16,18,29]
unknown = [8,9,17,24,30,99]


In [ ]:
# labelling the data
dest = pd.DataFrame([])
exit = pd.DataFrame([])
clientID = df.ClientID.unique()
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]] 
    if (temp['LivingSituation'].iloc[0] in stable)  :
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'stable'},ignore_index=True)
    elif temp['LivingSituation'].iloc[0] in unstable:
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'unstable'},ignore_index=True)
    elif temp['LivingSituation'].iloc[0] in unknown:
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'unknown'},ignore_index=True)
    
    if temp['Destination'].iloc[-1] in stable:
        exit = exit.append({"ClientID":clientID[i],'exit':"stable"},ignore_index=True)
    elif temp['Destination'].iloc[-1] in unstable:
            exit = exit.append({"ClientID":clientID[i],'exit':"unstable"},ignore_index=True)
    elif temp['Destination'].iloc[-1] in unknown:
            exit = exit.append({"ClientID":clientID[i],'exit':"unknown"},ignore_index=True)
    
            
data = pd.merge(dest,exit,on = "ClientID")
data.to_csv(path + "\\Scheme 2 labelled data.csv")

In [ ]:
# determining dropout probability trend

data = pd.read_csv(path + "\\Scheme 2 labelled data.csv").drop("Unnamed: 0", axis=1)

data_drop = pd.DataFrame()
for i in range(len(data)):
    for j in np.arange(data.PathLength.min(),data.PathLength.max()+1):
        if (j<data['PathLength'].iloc[i]):
            data_drop = data_drop.append({"ClientID":data['ClientID'].iloc[i], "PathLength": data['PathLength'].iloc[i],"DropOut":"reentry","step":j},ignore_index = True)
        elif (j==data['PathLength'].iloc[i]):
            if (data.iloc[i]['exit'] == "unstable") or (data.iloc[i]['exit'] == "unknown"):
                data_drop = data_drop.append({"ClientID":data['ClientID'].iloc[i], "PathLength": data['PathLength'].iloc[i],"DropOut":'dropout',"step":j},ignore_index = True)
            else:
                data_drop = data_drop.append({"ClientID":data['ClientID'].iloc[i], "PathLength": data['PathLength'].iloc[i],"DropOut":'success',"step":j},ignore_index = True)
        else:
            continue

loc = np.sort(data_drop["step"].unique())
data_drop_prob = pd.DataFrame()
for y in loc:
    temp = data_drop[data_drop['step'] == y]
    dropout = len(temp[temp["DropOut"]=='dropout'])/len(temp)
    not_dropoutOrReentry = len(temp[temp["DropOut"]=='success'])/len(temp) 
    reentry = len(temp[temp["DropOut"]=='reentry'])/len(temp) 
    data_drop_prob = data_drop_prob.append({"Path Position":y,"P(dropout)":dropout,"P(success)":not_dropoutOrReentry, "P(reentry)":reentry},ignore_index=True)

data_drop_prob['Dropout_MA'] = data_drop_prob.iloc[:,0].rolling(window=6).mean()
data_drop_prob['Reentry_MA'] = data_drop_prob.iloc[:,1].rolling(window=6).mean()
data_drop_prob['Success_MA'] = data_drop_prob.iloc[:,2].rolling(window=6).mean()

plt.figure(figsize = (20,10))
plt.plot(data_drop_prob['Path Position'],data_drop_prob['Dropout_MA'],marker = "x",linewidth = 2,label='Dropout')
plt.plot(data_drop_prob['Path Position'],data_drop_prob['Reentry_MA'],marker = "x",linewidth = 2,label='Reentry')
plt.plot(data_drop_prob['Path Position'],data_drop_prob['Success_MA'],marker = "x",linewidth = 2,label='Success')
plt.legend(fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Scheme 2",fontsize = 16)
plt.savefig(path+"\Scheme 2 Drop-out probability with reentry_original.png")

In [ ]:
# calculatin cdf

def cdf_plot(label,pdf,k):
    plt.subplot(1,3,k)
    size = 40
    min_x = 40
    max_x = 0
    target = ["stable","unstable","unknown"]
    index_tup=[(pdf.index.values[i][0],pdf.index.values[i][1]) for i in range(len(pdf.index.values))]    
    for i in target:
        if (label,i) not in index_tup:
            continue
        cdf = np.cumsum(pdf.loc[label,i]/pdf.loc[label,i].sum())
        cdf["ClientID"].plot(figsize = (30,10),marker="o",label=label+"-"+i,linewidth = 4)
        if cdf.index.min()<min_x:
            min_x = cdf.index.min()
        if cdf.index.max()>max_x:
            max_x = cdf.index.max()
    plt.legend(fontsize=size)
    plt.grid()
    # plt.yscale("symlog")
    # plt.xscale("symlog")
    plt.tight_layout()
    plt.xlabel('', fontsize=size)
    plt.yticks(fontsize=size)
    plt.xticks(np.arange(min_x,max_x,6),fontsize =size)

data = pd.read_csv(path + "\\Scheme 2 labelled data.csv").drop("Unnamed: 0", axis=1)
pdf = data.groupby(['entryFrom','exit','PathLength']).count()

initial = ["unstable","stable","unknown"]
[cdf_plot(initial[i],pdf,i+1) for i in range(len(initial))]
plt.savefig(path+"\\Scheme 2 Aggregated CDF.png", bbox_inches = "tight")